In [1]:
import numpy as np
import sys
sys.path.append("../")
import rms_torch 
import utils
import lattice

## Above procedure doesn't have kernel

In [2]:
import tensornetwork as tn
import numpy as np
from matplotlib import pyplot as plt
import copy 

In [12]:
sps = 8
bd = 2
AP = lattice.FF.block2D(sps, bd,seed=0)
peps, peps_edges = lattice.FF.create_PEPS(3,2,AP)
for k in range(len(peps_edges)):
    A = tn.contract(peps_edges[k])

### Demonstrate symmetry : translation, reflection

In [13]:
y1 = A.tensor.reshape(sps**2, - 1)
y1 = y1/np.linalg.norm(y1)
rho1 = y1 @ y1.T

y2 = A.tensor.transpose(1,0,2,3,4,5)
y2 = y2/np.linalg.norm(y2)
rho2 = np.einsum("ijabcd, klabcd->ijkl",y2, y2).reshape(sps**2, sps**2)

np.linalg.norm(rho1 - rho2)

6.019113015731008e-16

In [14]:
A.tensor[0,1,2,2,1,0], A.tensor[1,2,0,1,0,2], A.tensor[2,1,0,0,1,2]

(875.7768813175701, 875.7768813175701, 875.7768813175701)

### Generate hamiltonian and check symmetry

In [15]:
AP2 = np.einsum("ijklm,abide->mejklabd",AP,AP).reshape(sps ** 2, -1)
U, s, V = np.linalg.svd(AP2)
s = np.round(s,10)
s = s[s!=0]
Up = U[:, len(s):]
h1 = Up @ Up.T

In [16]:
AP2 = np.einsum("ijkbm,abcde->meijkacd",AP,AP).reshape(sps ** 2, -1)
U, s, V = np.linalg.svd(AP2)
s = np.round(s,10)
s = s[s!=0]
Up = U[:, len(s):]
h2 = Up @ Up.T
print(np.linalg.norm(h1 - h2))

1.0063678396838812e-14


In [17]:
np.linalg.norm(h1.reshape((sps,)*4).transpose(1,0,3,2) - h1.reshape((sps,)*4))

9.014831804188498e-15

In [18]:
y = A.tensor.reshape(sps**2, - 1)
y = y/np.linalg.norm(y)
rho = y @ y.T
print(np.trace(rho @ h1))

y = A.tensor.transpose(0,3,1,4,5,2)
y = y/np.linalg.norm(y)
rho = np.einsum("ijabcd, klabcd->ijkl",y, y).reshape(sps**2, sps**2)
print(np.trace(rho @ h1))

-1.776915728899653e-17
-2.6791171260048176e-17
